In [1]:
import numpy 
import scipy.special
import matplotlib.pyplot
%matplotlib inline 

In [2]:
class neural_network:
    
    def __init__(self,inputnodes, hiddennodes, outputnodes, learningrate):
        self.inodes = inputnodes
        self.hnodes = hiddennodes
        self.onodes = outputnodes
        
        self.wih = numpy.random.normal(0.0, pow(self.hnodes, -0.5),(self.hnodes, self.inodes))
        self.who = numpy.random.normal(0.0, pow(self.onodes, -0.5),(self.onodes,self.hnodes))
        
        self.lr = learningrate
        
        self.activation_function = lambda x: scipy.special.expit(x)
        
        pass
    
      # train the neural network
    def train(self, inputs_list, targets_list):
    # convert inputs list to 2d array
        inputs = numpy.array(inputs_list, ndmin=2).T
        targets = numpy.array(targets_list, ndmin=2).T
    # calculate signals into hidden layer
        hidden_inputs = numpy.dot(self.wih, inputs)
    # calculate the signals emerging from hidden layer
        hidden_outputs = self.activation_function(hidden_inputs)
    # calculate signals into final output layer
        final_inputs = numpy.dot(self.who, hidden_outputs)
    # calculate the signals emerging from final output layer
        final_outputs = self.activation_function(final_inputs)
    # output layer error is the (target actual)
        output_errors = targets-final_outputs
    # hidden layer error is the output_errors, split by weights,recombined at hidden nodes
        hidden_errors = numpy.dot(self.who.T, output_errors)
    # update the weights for the links between the hidden and output layers
        self.who += self.lr * numpy.dot((output_errors * final_outputs * (1.0-final_outputs)),numpy.transpose(hidden_outputs))
    # update the weights for the links between the input and hidden layers
        self.wih += self.lr * numpy.dot((hidden_errors *hidden_outputs * (1.0-hidden_outputs)), numpy.transpose(inputs))
    pass
    
    def query(self, inputs_list):
    # convert inputs list to 2d array
        inputs = numpy.array(inputs_list, ndmin=2).T
        # calculate signals into hidden layer
        hidden_inputs = numpy.dot(self.wih, inputs)
        # calculate the signals emerging from hidden layer
        hidden_outputs = self.activation_function(hidden_inputs)
        # calculate signals into final output layer
        final_inputs = numpy.dot(self.who, hidden_outputs)
        # calculate the signals emerging from final output layer
        final_outputs = self.activation_function(final_inputs)
        return final_outputs

In [3]:
input_nodes=784
hidden_nodes= 100
output_nodes = 10
learning_rate = 0.2

n= neural_network(input_nodes,hidden_nodes,output_nodes,learning_rate)

#MNIST dataset 
train_data_file = open("mnist_train.csv", 'r')
train_data_list = train_data_file.readlines()
train_data_file.close()

#adding epochs
epoch=5 
#Training the Neural network 
for e in range(epoch):
    for record in train_data_list:
        all_values = record.split(',')

        #Scaling and shifting of the inputs
        inputs = (numpy.asfarray(all_values[1:])/255.0 * 0.99) + 0.01

        #0-9 setting the target valuse of the node, from MNIST dataset chosing the first cell and setting that number to have prob 0.99
        targets = numpy.zeros(output_nodes)+0.01
        targets[int(all_values[0])]=0.99


        n.train(inputs,targets)
        pass
    pass

In [4]:
#test the neural network
test_data_file= open("mnist_test.csv",'r')
test_data_list= test_data_file.readlines()
test_data_file.close()

#Scorecard
scorecard=[]
sum=0
all_values=test_data_list[0].split(',')
n.query((numpy.asfarray(all_values[1:]) / 255.0 * 0.99) + 0.01)


array([[0.00790319],
       [0.02096725],
       [0.00821454],
       [0.05677263],
       [0.02779267],
       [0.00786807],
       [0.0288586 ],
       [0.98350549],
       [0.00586498],
       [0.03841602]])

In [5]:
for record in test_data_list:
    #splitting the comma seperated values
    all_values = record.split(',')
    #Correct answer is the 0th order valuw of the data set
    correct_label=int(all_values[0])
    #print("Correct answer=",correct_label)
    
    inputs = (numpy.asfarray(all_values[1:])/255.0 * 0.99)+0.01

    #query the network
    outputs= n.query(inputs)
  
    #highest value is the label predicted by network
    label=numpy.argmax(outputs)
    #print('network answer=',label)
    #appending correct or incorrect to the scorecard 
    if(label==correct_label):
        scorecard.append(1)
        sum=sum+1
    else:
        scorecard.append(0)
        pass
    pass
scorecard_array = numpy.asarray(scorecard)
print ("performance = ", scorecard_array.sum() / scorecard_array.size)
 

performance =  0.9601


In [31]:
import imageio
img_array = imageio.imread ( "1bpng.png", as_gray = True )
img_data = 255.0 - img_array.reshape(784)
img_data = (img_data / 255.0 * 0.99 ) + 0.01

In [32]:
print(numpy.argmax(n.query(img_data)))

3
